In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
%pip install semantic-link-labs --q

In [3]:
import sempy_labs as labs
import sempy_labs.admin as labs_admin
from pyspark.sql import Row
import datetime

StatementMeta(, b4c5e0ae-5720-485f-a76f-801187d1dfb4, 12, Finished, Available, Finished)

In [ ]:
def log_to_table(level, message):
    now = datetime.datetime.utcnow().isoformat()
    log_df = spark.createDataFrame([Row(timestamp=now, level=level, message=message)])
    log_df.write.format("delta").mode("append").saveAsTable("semanticModel_conversion_toSmall")

StatementMeta(, b4c5e0ae-5720-485f-a76f-801187d1dfb4, 13, Finished, Available, Finished)

In [ ]:
current_user = mssparkutils.env.getUserName()
workspaces_df = spark.sql(f"Select DISTINCT Dataset_Id ,Dataset_Name, Workspace_Id from models_discovery_status where Status <> 'Success' and Size > 0 ")
log_to_table("INFO", f"Format conversion started.")
for row in workspaces_df.collect():
    ws_id = row['Workspace_Id']
    ds_id = row['Dataset_Id']
    ds_name = row['Dataset_Name']
    try:
      #labs_admin.add_user_to_workspace(workspace=ws_id, user=current_user, role="Admin")
      labs.set_semantic_model_storage_format(dataset=ds_id, storage_format='Small', workspace=ws_id)
      print(f" -Format converted successfully.'{ds_name}'.")
      log_to_table("INFO", f"Format converted successfully. {ds_name}.")
      #labs_admin.delete_user_from_workspace(workspace=ws_id, user=current_user)
    except Exception as e:
        print(f"  ❌ FAILED: An error occurred while processing model {ds_name}.")
        print(f"    Error details: {e}")
        log_to_table("ERROR", f"Exception occurred: in dataset {ds_name} error: {str(e)}")
log_to_table("INFO", f"Format conversion Completed.")